## Big Mart Sales  - Prediction

### The Problem Statement
The data scientists at BigMart have collected 2013 sales data for 1559 products across 10 stores in different cities. Also, certain attributes of each product and store have been defined. The aim is to build a predictive model and find out the sales of each product at a particular store.

Using this model, BigMart will try to understand the properties of products and stores which play a key role in increasing sales

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from BigMartdata import getcsv_data

In [3]:
dataset= getcsv_data()

Train dataframe :- (8523, 13)
Test dataframe :- (5681, 12)
Combined dataset dataframe :- (14204, 13)


In [4]:
from BigMartdata import getcsv_train

In [5]:
train = getcsv_train()

In [6]:
train.shape

(8523, 12)

In [ ]:
dataset.apply(lambda x: sum(x.isnull()))

In [ ]:
# Alternatively use th following function
dataset.isnull().sum()

In [ ]:
dataset.describe()

In [ ]:
dataset.apply(lambda x: len(x.unique()))

In [ ]:
#Filter categorical variables
categorical_columns = [x for x in dataset.dtypes.index if dataset.dtypes[x]=='object']
categorical_columns

In [ ]:
# Alternatively use th following function
cat_cols = []
for i in dataset.dtypes.index :
    if dataset.dtypes[i]=='object' :
        cat_cols.append(i)
cat_cols

In [ ]:
#Exclude Identifier columns and source column which was created newly:
categorical_columns = [x for x in categorical_columns if x not in ['Item_Identifier','Outlet_Identifier','source']]
categorical_columns

In [ ]:
#Print frequency of categories
for col in categorical_columns:
    print ('Frequency of Categories for varible : ', col)
    print (dataset[col].value_counts())
    print('---------------------------------------------------')

The output gives us following observations:

> Item_Fat_Content: Some of ‘Low Fat’ values mis-coded as ‘low fat’ and ‘LF’. Also, some of ‘Regular’ are mentioned as ‘regular’. <br>
Item_Type: Not all categories have substantial numbers. It looks like combining them can give better results. <br>
Outlet_Type: Supermarket Type2 and Type3 can be combined. But we should check if that’s a good idea before doing it.

### 3. Data Cleaning
> Imputing missing values and Treating outliers

#### a. Item_Weight with the help of Item_Identifier

In [ ]:
dataset.apply(lambda x: sum(x.isnull()))

Out of the independet values, we can see that Item_Weight and Outlet_Size columns have missing values

In [ ]:
#Determine the average weight per item:
Item_ave_weight = dataset.pivot_table(values='Item_Weight', index='Item_Identifier')
Item_ave_weight.head()

In [ ]:
#Get a boolean variable specifying missing Item_Weight values
miss_IW_rows = dataset['Item_Weight'].isnull() # Output is a list of boolean values
miss_IW_rows.head()

In [ ]:
dataset.loc[miss_IW_rows,'Item_Weight'] = dataset.loc[miss_IW_rows,'Item_Identifier'].apply(lambda x: Item_ave_weight.loc[x])    

In [ ]:
print ('Orignal #missing values:', sum(miss_IW_rows))
print ('Final #missing values:', sum(dataset['Item_Weight'].isnull()))

#### b. Outlet_Size with the help of Outlet_Type

In [ ]:
#### a. Item_Weight with the help of Item_Identifier#Import mode function:
from statistics import mode
#Determining the mode for each
Item_Outlet_size = dataset.pivot_table(values='Outlet_Size', index='Outlet_Type',aggfunc=(lambda x:mode(x)))
Item_Outlet_size

In [ ]:
dataset.Outlet_Type.unique()

As the mode parameter is not indicating the 'Grocery Store' value, <br> we will need to 'physically' impute the Outlet_Size as 'Small' to Outlet_Type 'Grocery Store'.

In [ ]:
miss_OS_GS_rows = dataset [(dataset['Outlet_Type'] == 'Grocery Store') & (dataset['Outlet_Size'].isnull())].index

In [ ]:
dataset.loc[miss_OS_GS_rows,'Outlet_Size'] = 'Small'

In [ ]:
miss_OS_rows = dataset['Outlet_Size'].isnull()

In [ ]:
dataset.loc[miss_OS_rows,'Outlet_Size'] = dataset.loc[miss_OS_rows,'Outlet_Type'].apply(lambda x: Item_Outlet_size.loc[x])

In [ ]:
print ('Original missing : ', sum(miss_OS_rows))
print ('After update missing : ' , sum(dataset['Outlet_Size'].isnull()))

### 4.0 Feature Engineering

Considering to __combine values__ in column __Outlet_Type__

In [ ]:
dataset.head(10)

In [ ]:
 dataset.groupby(['Outlet_Type'])['Item_Outlet_Sales'].sum()

In [ ]:
dataset.pivot_table(values='Item_Outlet_Sales',index='Outlet_Type')

The mean outlet sales values *vary significantly* and hence we will leave them as it is.

####  Item_Visibility with the help of Item_Identifier

In [ ]:
dataset[dataset['Item_Visibility'] == 0].head()

In [ ]:
Item_ave_visi = dataset.pivot_table(values='Item_Visibility', index='Item_Identifier')

In [ ]:
Item_ave_visi.head()

In [ ]:
miss_val_ItemVis = (dataset['Item_Visibility'] == 0)

In [ ]:
dataset.loc[miss_val_ItemVis,'Item_Visibility'] = dataset.loc[miss_val_ItemVis,'Item_Identifier'].apply(lambda x: Item_ave_visi.loc[x])

In [ ]:
print ('Number of 0 values in Item_Visibility initially:' , sum (miss_val_ItemVis))
print ('Number of 0 values in Item_Visibility after impute:' , sum (dataset['Item_Visibility'] == 0))

In [ ]:
#Determine another variable with means ratio
dataset['Item_Visibility_MeanRatio'] = dataset.apply(lambda x: x['Item_Visibility']/Item_ave_visi.loc[x,'Item_Visibility'], axis=1)

#### Create a broad category of Type of Item

In [ ]:
# Get the first two characters of ID:
dataset['Item_Type_Combined'] = dataset['Item_Identifier'].apply(lambda x: x[0:2])

In [ ]:
dataset['Item_Type_Combined'].unique()

In [ ]:
# Rename them to more intuitive categories:
dataset['Item_Type_Combined'] = dataset['Item_Type_Combined'].map(
    {'FD':'Food','NC':'Non-Consumable','DR':'Drinks'})

In [ ]:
dataset['Item_Type_Combined'].value_counts()

#### Determine the years of operation of a store

In [ ]:
#Years in operation :
import datetime
now = datetime.datetime.now()
dataset['Outlet_Since_Years'] = now.year - dataset['Outlet_Establishment_Year']

#### Modify categories of Item_Fat_Content

In [ ]:
#Change categories of low fat:
print ('Original Item_Fat_Content Categories:' ) 
print (dataset['Item_Fat_Content'].value_counts())

In [ ]:
# Rename Item_Fat_Content to proper names (consider upper case also)
dataset['Item_Fat_Content'] = dataset['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
print ('Item_Fat_Content Categories after renaming values :' ) 
print (dataset['Item_Fat_Content'].value_counts())

In [ ]:
#Marking non-consumables as separate category in Item_Fat_Content:
dataset.loc[dataset['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
dataset['Item_Fat_Content'].value_counts()

#### Encoding of categorical variables to numericals

In [ ]:
#Import library:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
dataset.columns

In [ ]:
dataset.dtypes

In [ ]:
#New variable for outlet
dataset['Outlet'] = le.fit_transform(dataset['Outlet_Identifier'])

In [ ]:
dataset[['Outlet_Identifier', 'Outlet']].head()

In [ ]:
cat_cols1 = []
for i in dataset.dtypes.index :
    if dataset.dtypes[i]=='object' :
        cat_cols1.append(i)
cat_cols1

In [ ]:
# Adding varaibles Outlet to the list and excluding identifiers and Item_Type
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']

In [ ]:
# Encoding categorical variables to numerics based on the above list (var_mod) created
le_x = LabelEncoder()
for i in var_mod:
    dataset[i] = le_x.fit_transform(dataset[i])

In [ ]:
dataset[['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']].head()

#### One Hot encoding of variables

> One-Hot-Coding refers to creating dummy variables, one for each category of a categorical variable. 

>For example, the Item_Fat_Content has 3 categories – ‘Low Fat’, ‘Regular’ and ‘Non-Edible’. <br> 
One hot coding __will remove__ this variable __and generate__ 3 __new variables__. Each will have binary numbers – 0 (if the category is not present) and 1(if category is present). 

>This can be done using ‘get_dummies’ function of Pandas.

In [ ]:
dataset = pd.get_dummies(dataset, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                              'Item_Type_Combined','Outlet'])

In [ ]:
dataset.dtypes

In [ ]:
dataset[['Item_Fat_Content_0','Item_Fat_Content_1','Item_Fat_Content_2']].head(10)

#### Exporting the data

In [ ]:
dataset[['Item_Type','Item_Type_Combined_0', 'Outlet_Establishment_Year', 'Outlet_Since_Years']].head()
# Item type --> A new column 'Item_Type_Combined' was created mainly to obtain more counts in this categorical 
# varlaible, and was given intutive names based on the codes. The Item_Type column is 'repetition' of Item_Type_Combined.

# Outlet_Establishment_Year --> A new column 'Outlet_Since_Years' was created using 'Outlet_Establishment_Year' column 

In [ ]:
# Drop the columns which have been converted to different types:
dataset.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)

In [ ]:
# Splitting the dataset 'back' into test and train datasets:
train = dataset.loc[dataset['source']=="train"]
test = dataset.loc[dataset['source']=="test"]

In [ ]:
# Dropping unnecessary columns:
test = test.drop(['Item_Outlet_Sales','source'],axis=1) # dropping dependent variable 'Item_Outlet_Sales' from this dataset
train = train.drop(['source'],axis=1)

In [ ]:
test.columns

In [ ]:
train.columns

In [ ]:
# Exporting csv files as modified versions:
train.to_csv("train_modified.csv",index=False)
test.to_csv("test_modified.csv",index=False)

### 4. Building the model

In [ ]:
# Mean based:
mean_sales = train['Item_Outlet_Sales'].mean()
mean_sales

In [ ]:
# Defining a dataframe with IDs for submission:
base1 = test[['Item_Identifier','Outlet_Identifier']]

In [ ]:
# Implying the mean sales value to the Item_Outlet_Sales column
base1.loc[:,'Item_Outlet_Sales'] = mean_sales

In [ ]:
# Exporting the submission file
base1.to_csv("alg0.csv",index=False)

__Function to automate submission to Hackothon__:
> A generic function which takes the algorithm and data as input and makes the model, performs cross-validation and generates submission

In [ ]:
#Define target and ID columns:

target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet_Identifier']

from sklearn.model_selection import cross_val_score
from sklearn import metrics
def modelfit(algorthm, dftrain, dftest, predictors, target, IDcol, filename):

    #Fit the algorthmorithm on the data
    # Creating ID columns and target columns in the new vriable
    algorthm.fit(dftrain[predictors], dftrain[target]) # similar to the base dataframe created above with the predictor & target columns
        
    #Predict training set:
    dftrain_predictions = algorthm.predict(dftrain[predictors]) # creating new predictor column 

    #Perform cross-validation:
    cv_score = cross_val_score(algorthm, dftrain[predictors], dftrain[target], cv=20, scoring='neg_mean_squared_error')
    cv_score = np.sqrt(np.abs(cv_score))
    
    #Print model report:
    print ("\n------Model Report----\n")
    print ("RMSE : " , np.sqrt(metrics.mean_squared_error(dftrain[target].values, dftrain_predictions)))
    print ("CV Score Mean : %.4g" %(np.mean(cv_score)))
    print ("CV Score Std : %.4g" %(np.std(cv_score)))
    print ("CV Score Min : %.4g" %(np.min(cv_score)))
    print ("CV Score Max : %.4g" %(np.max(cv_score)))
    
    #Predict on testing data:
    dftest[target] = algorthm.predict(dftest[predictors])
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dftest[x] for x in IDcol})
    submission.to_csv(filename, index=False)

### Linear Regression Model

In [ ]:
train.head()

In [ ]:
# Building a list of independent varaibles to build the model
target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet_Identifier']

# Excluding Identifiers (used for submission) and dependent variable column
predictors = [x for x in train.columns if x not in [target]+IDcol]
predictors

In [ ]:
# Building a linear regression model 
from sklearn.linear_model import LinearRegression, Ridge, Lasso
alg1 = LinearRegression(normalize=True)

In [ ]:
# Using the FUNCTION which takes the algorithm and data as input and makes the model 
modelfit(alg1, train, test, predictors, target, IDcol, 'alg1.csv')

In [ ]:
coef1 = pd.Series(alg1.coef_, predictors).sort_values(ascending=True)
coef1

In [ ]:
plt.figure(figsize=(8,6))
coef1.plot(kind='bar', title='Model Coefficients')
plt.show()

### Ridge Regression Model:

In [ ]:
# Building a list of independent varaibles to build the model
target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet_Identifier']
predictors = [x for x in train.columns if x not in [target]+IDcol]

In [ ]:
# Building a Ridge regression model 
alg2 = Ridge(alpha=0.05,normalize=True)

In [ ]:
modelfit(alg2, train, test, predictors, target, IDcol, 'alg2.csv')
coef2 = pd.Series(alg2.coef_, predictors).sort_values(ascending=True)
plt.figure(figsize=(8,6))
coef2.plot(kind='bar', title='Model Coefficients')
plt.show()

In [ ]:
coef2

### Decision Tree Model

In [ ]:
# Building a list of independent varaibles to build the model
target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet_Identifier']
predictors = [x for x in train.columns if x not in [target]+IDcol]

In [ ]:
# Building a Decision Tree regression model 
from sklearn.tree import DecisionTreeRegressor
alg3 = DecisionTreeRegressor(max_depth=15, min_samples_leaf=100)


In [ ]:
modelfit(alg3, train, test, predictors, target, IDcol, 'alg3.csv')
coef3 = pd.Series(alg3.feature_importances_, predictors).sort_values(ascending=False)
plt.figure(figsize=(8,6))
coef3.plot(kind='bar', title='Feature Importances')
plt.show()

In [ ]:
coef3

Here you can see that the RMSE is 1058 and the mean CV error is 1091. <br>
This tells us that the __model is slightly overfitting__. <br> 
> Lets try making a decision tree with just top 4 variables, a max_depth of 8 and min_samples_leaf as 150.

In [ ]:
# Building a list of independent varaibles to build the model
target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet_Identifier']
# Making a decision tree with just **top 4** variables based on the above coeffecient values
predictors = ['Item_MRP','Outlet_Type_0', 'Outlet_5', 'Outlet_Since_Years']

In [ ]:
# Building a Decision Tree regression model 
# Considering a max_depth of 8 and min_samples_leaf as 150.
from sklearn.tree import DecisionTreeRegressor
alg4 = DecisionTreeRegressor(max_depth=8, min_samples_leaf=150)

In [ ]:
modelfit(alg4, train, test, predictors, target, IDcol, 'alg4.csv')
coef4 = pd.Series(alg4.feature_importances_, predictors).sort_values(ascending=False)
plt.figure(figsize=(8,6))
coef4.plot(kind='bar', title='Feature Importances')
plt.show()

Other parameters can be fine tuned to bring down the overfitting

### Random Forest Model

In [ ]:
# Building a list of independent varaibles to build the model
target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet_Identifier']
predictors = [x for x in train.columns if x not in [target]+IDcol]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
alg5 = RandomForestRegressor(n_estimators=200,max_depth=5, min_samples_leaf=100,n_jobs=4)

In [ ]:
modelfit(alg5, train, test, predictors, target, IDcol, 'alg5.csv')
coef5 = pd.Series(alg5.feature_importances_, predictors).sort_values(ascending=False)
plt.figure(figsize=(8,6))
coef5.plot(kind='bar', title='Feature Importances')
plt.show()

You might feel this is a very small improvement but as our model gets better, achieving even minute improvements becomes exponentially difficult. <br>
>Lets try another random forest with max_depth of 6 and 400 trees. Increasing the number of trees makes the model robust but is computationally expensive.

In [ ]:
# Building Random Forest model with max_depth of 6 and 400 trees
from sklearn.ensemble import RandomForestRegressor
alg6 = RandomForestRegressor(n_estimators=400,max_depth=6, min_samples_leaf=100,n_jobs=4)

In [ ]:
modelfit(alg6, train, test, predictors, target, IDcol, 'alg6.csv')
coef6 = pd.Series(alg6.feature_importances_, predictors).sort_values(ascending=False)
plt.figure(figsize=(8,6))
coef6.plot(kind='bar', title='Feature Importances')
plt.show()

__End Notes__<br>
This article took us through the entire journey of solving a data science problem. We started with making some hypothesis about the data without looking at it. Then we moved on to data exploration where we found out some nuances in the data which required remediation. Next, we performed data cleaning and feature engineering, where we imputed missing values and solved other irregularities, made new features and also made the data model-friendly by one-hot-coding. Finally we made regression, decision tree and random forest model and got a glimpse of how to tune them for better results.

I believe everyone reading this article should attain a good score in BigMart Sales now. For beginners, you should achieve at least a score of 1150 and for the ones already on the top, you can use some feature engineering tips from here to go further up. All the best to all!

Did you find this article useful? Could you make some more interesting hypothesis? What other features did you create? Were you able to get a better score with GBM & XGBoost? Feel free to discuss your experiences in comments below or on the discussion portal and we’ll be more than happy to discuss.